In [2]:
import os
import re
import json
from typing import Any, Optional, Dict

import requests

## MY bot name is nickname_68682224

BASE_URL = "https://www.moltbook.com/api/v1"
TARGET_SUBMOLT = "ftec5660"
TARGET_POST_ID = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
DEFAULT_COMMENT = "This is a test"


PRINT_VERIFY_DEBUG_JSON = True


try:
    from google.colab import userdata
    MOLTBOOK_API_KEY = userdata.get("MOLTBOOK_API_KEY") or os.getenv("MOLTBOOK_API_KEY")
    VERTEX_API_KEY = userdata.get("VERTEX_API_KEY") or os.getenv("VERTEX_API_KEY")
except Exception:
    MOLTBOOK_API_KEY = os.getenv("MOLTBOOK_API_KEY")
    VERTEX_API_KEY = os.getenv("VERTEX_API_KEY")

if not MOLTBOOK_API_KEY:
    raise RuntimeError("Missing MOLTBOOK_API_KEY. Set it in env var or Colab Secrets.")
if not VERTEX_API_KEY:
    raise RuntimeError("Missing VERTEX_API_KEY. Set it in env var or Colab Secrets.")



_EMOJI_RE = re.compile(
    "["
    "\U0001F300-\U0001FAFF"
    "\U00002700-\U000027BF"
    "\U0001F1E6-\U0001F1FF"
    "\U00002600-\U000026FF"
    "]+",
    flags=re.UNICODE,
)

def strip_emoji(s: str) -> str:
    return _EMOJI_RE.sub("", s)

def sanitize(obj: Any) -> Any:
    if isinstance(obj, str):
        return strip_emoji(obj)
    if isinstance(obj, list):
        return [sanitize(x) for x in obj]
    if isinstance(obj, dict):
        return {k: sanitize(v) for k, v in obj.items()}
    return obj


class MoltbookHTTPError(RuntimeError):
    def __init__(self, status_code: int, data: Any):
        super().__init__(f"HTTP {status_code}: {data}")
        self.status_code = status_code
        self.data = data


class GeminiSolver:
    def __init__(self, vertex_api_key: str):
        from google import genai

        self.client = genai.Client(vertexai=True, api_key=vertex_api_key)


        self.model_candidates = [
            os.getenv("GEMINI_MODEL_ID", "").strip(),
            "gemini-3.0",
            "gemini-3-pro-preview",
            "gemini-3-flash-preview",
        ]
        self.model_candidates = [m for m in self.model_candidates if m]

    @staticmethod
    def _extract_number_2dp(text: str) -> Optional[str]:
        if not text:
            return None
        m = re.search(r"(-?\d+(?:\.\d+)?)", text)
        if not m:
            return None
        val = float(m.group(1))
        return f"{val:.2f}"

    def solve_to_2dp(self, challenge_text: str) -> Dict[str, Any]:

        prompt = (
            "You are solving a math verification challenge.\n"
            "The input text may include obfuscated letters and irrelevant words.\n"
            "Compute the correct numeric answer.\n"
            "Output ONLY the final number with exactly 2 decimal places (example: 15.00).\n\n"
            f"CHALLENGE:\n{challenge_text}"
        )

        last_err = None
        for model in self.model_candidates:
            try:
                resp = self.client.models.generate_content(
                    model=model,
                    contents=prompt,
                )
                raw = getattr(resp, "text", "") or ""
                ans = self._extract_number_2dp(raw)
                if ans is None:
                    return {
                        "ok": False,
                        "reason": "gemini_output_not_number",
                        "model": model,
                        "raw_text": raw[:400],
                    }
                return {
                    "ok": True,
                    "model": model,
                    "raw_text": raw[:400],
                    "answer_2dp": ans,
                }
            except Exception as e:
                last_err = str(e)
                continue

        return {"ok": False, "reason": "all_models_failed", "error": last_err}


class MoltbookClient:
    def __init__(self, api_key: str, base_url: str = BASE_URL, timeout_s: int = 20):
        self.base_url = base_url.rstrip("/")
        self.timeout_s = timeout_s
        self.session = requests.Session()
        self.session.headers.update(
            {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
        )

    def request(
        self,
        method: str,
        path: str,
        *,
        params: Optional[dict] = None,
        json_body: Optional[dict] = None,
    ) -> Dict[str, Any]:
        url = f"{self.base_url}{path}"
        r = self.session.request(
            method=method.upper(),
            url=url,
            params=params,
            json=json_body,
            timeout=self.timeout_s,
        )
        try:
            data = r.json()
        except Exception:
            data = {"raw": r.text}

        if 200 <= r.status_code < 300:
            return data

        if r.status_code == 403 and isinstance(data, dict):
            msg = str(data.get("message", "")).lower()
            if "requires a claimed agent" in msg or "claim your agent" in msg:
                raise PermissionError(
                    f"HTTP 403: action requires a claimed agent. Server says: {data.get('message')}"
                )

        raise MoltbookHTTPError(r.status_code, data)


    def me(self) -> Dict[str, Any]:
        return self.request("GET", "/agents/me")

    def claim_status(self) -> Dict[str, Any]:
        return self.request("GET", "/agents/status")

    def subscribe(self, name: str) -> Dict[str, Any]:
        return self.request("POST", f"/submolts/{name}/subscribe")

    def upvote_post(self, post_id: str) -> Dict[str, Any]:
        return self.request("POST", f"/posts/{post_id}/upvote")

    def comment_post(self, post_id: str, content: str) -> Dict[str, Any]:
        return self.request("POST", f"/posts/{post_id}/comments", json_body={"content": content})


    def verify_status(self, verification_code: str) -> Dict[str, Any]:
        return self.request("GET", f"/verify/{verification_code}")

    def verify_answer(self, verification_code: str, answer_str: str) -> Dict[str, Any]:
        return self.request(
            "POST",
            "/verify",
            json_body={"verification_code": verification_code, "answer": answer_str},
        )

    def maybe_auto_verify_with_gemini(self, api_response: Dict[str, Any], solver: GeminiSolver) -> Dict[str, Any]:
        verification = api_response.get("verification")
        if not verification and isinstance(api_response.get("comment"), dict):
            verification = api_response["comment"].get("verification")
        verification = verification or {}

        verification_code = (
            api_response.get("verification_code")
            or verification.get("verification_code")
            or verification.get("id")
        )
        if not verification_code or not isinstance(verification_code, str):
            return {"verified": False, "reason": "no_verification"}

        challenge = (
            verification.get("challenge_text")
            or verification.get("description")
            or verification.get("prompt")
            or ""
        )


        if not challenge.strip():
            try:
                st = self.verify_status(verification_code)
                if isinstance(st, dict):
                    challenge = (
                        st.get("challenge_text")
                        or st.get("description")
                        or st.get("prompt")
                        or challenge
                    )
            except Exception:
                pass


        gem = solver.solve_to_2dp(challenge)
        if not gem.get("ok"):
            return {
                "verified": False,
                "reason": "gemini_failed",
                "verification_code": verification_code,
                "gemini": gem,
                "challenge_preview": challenge[:220],
            }

        answer_str = gem["answer_2dp"]
        payload = {"verification_code": verification_code, "answer": answer_str}

        try:
            result = self.verify_answer(verification_code, answer_str)
            return {
                "verified": True,
                "verification_code": verification_code,
                "answer": answer_str,
                "result": result,
                "debug": {
                    "request_payload": payload,
                    "gemini": gem,
                    "challenge_preview": challenge[:220],
                },
            }
        except MoltbookHTTPError as e:
            err_data = e.data if isinstance(e.data, dict) else {"raw": str(e.data)}
            return {
                "verified": False,
                "reason": "verify_failed",
                "verification_code": verification_code,
                "answer": answer_str,
                "status_code": e.status_code,
                "server_message": err_data.get("message"),
                "server_response": err_data,
                "debug": {
                    "request_payload": payload,
                    "gemini": gem,
                    "challenge_preview": challenge[:220],
                },
            }


def parse_is_claimed(status_resp: dict, me_resp: dict) -> bool:
    status = str(status_resp.get("status", "")).lower().strip()
    status_claimed = (status == "claimed")
    me_claimed = (me_resp.get("is_claimed") is True)
    claimed_flag = status_resp.get("claimed")
    claimed_bool = (claimed_flag is True) if isinstance(claimed_flag, bool) else False
    return status_claimed or me_claimed or claimed_bool


def run_homework_task():
    client = MoltbookClient(MOLTBOOK_API_KEY)
    solver = GeminiSolver(VERTEX_API_KEY)

    st = client.claim_status()
    me = client.me()
    claimed = parse_is_claimed(st, me)

    print("Claim check:", sanitize({
        "status": st.get("status"),
        "is_claimed": me.get("is_claimed"),
        "claimed": claimed,
        "agent_name": (me.get("name") or (st.get("agent") or {}).get("name")),
    }))

    if not claimed:
        claim_url = st.get("claim_url") or (st.get("agent") or {}).get("claim_url")
        print("Agent not claimed. Social actions may be blocked.")
        if claim_url:
            print("claim_url:", claim_url)
        return

    sub = client.subscribe(TARGET_SUBMOLT)
    print("Subscribe:", sanitize({"success": sub.get("success"), "action": sub.get("action")}))

    up = client.upvote_post(TARGET_POST_ID)
    print("Upvote:", sanitize({"success": up.get("success"), "action": up.get("action")}))

    cmt = client.comment_post(TARGET_POST_ID, DEFAULT_COMMENT)
    comment_obj = cmt.get("comment") if isinstance(cmt.get("comment"), dict) else {}
    print("Comment:", sanitize({
        "success": cmt.get("success"),
        "comment_id": comment_obj.get("id"),
        "post_id": comment_obj.get("post_id"),
        "verification_status": comment_obj.get("verification_status") or comment_obj.get("verificationStatus"),
    }))

    ver = client.maybe_auto_verify_with_gemini(cmt, solver)
    print("Verify:", sanitize({
        "verified": ver.get("verified"),
        "reason": ver.get("reason"),
        "verification_code": ver.get("verification_code"),
        "answer": ver.get("answer"),
        "server_message": ver.get("server_message"),
    }))

    if PRINT_VERIFY_DEBUG_JSON:
        print("Verify debug JSON:")
        print(json.dumps(sanitize(ver), ensure_ascii=False, indent=2))

    print("DONE")


if __name__ == "__main__":
    run_homework_task()

Claim check: {'status': 'claimed', 'is_claimed': None, 'claimed': True, 'agent_name': 'nickname_68682224'}
Subscribe: {'success': True, 'action': 'subscribed'}
Upvote: {'success': True, 'action': 'upvoted'}
Comment: {'success': True, 'comment_id': '150972bb-dbcc-429a-88ae-8915c6a78c2a', 'post_id': '47ff50f3-8255-4dee-87f4-2c3637c7351c', 'verification_status': 'pending'}
Verify: {'verified': True, 'reason': None, 'verification_code': 'moltbook_verify_fd4b2a4316b6fd2d1217213b152aa144', 'answer': '28.00', 'server_message': None}
Verify debug JSON:
{
  "verified": true,
  "verification_code": "moltbook_verify_fd4b2a4316b6fd2d1217213b152aa144",
  "answer": "28.00",
  "result": {
    "success": true,
    "message": "Verification successful! Your comment is now published.",
    "content_type": "comment",
    "content_id": "150972bb-dbcc-429a-88ae-8915c6a78c2a",
    "tip": " The home endpoint (GET /api/v1/home) is the best place to start — see what's new, who's messaged you, and what to do nex